In [ ]:
pip install beautifulsoup4

In [ ]:
pip install requests

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

In [ ]:
sozluk = input("Lutfen kelimeyi giriniz: ")
sozluk = sozluk.replace(" ", "-")
sozluk = sozluk.lower()

Lutfen kelimeyi giriniz: kırmızı


# Kelimenin anlamlarını sözlükten alma

In [ ]:
def AranilanKelimeninAnlamlari(sozluk):
  sozlukurl = ("https://nedirara.com/{}".format(sozluk))
  try:
    r = requests.get(sozlukurl)
    soup = BeautifulSoup(r.content, "html")
  except requests.exceptions.ConnectionError as ex:
      print('Issue file download...' + str(ex))
  except Exception as ex:
      print('Issue file download...' + str(ex))

  anlamlari = soup.find_all("div", attrs = {"class": "panel panel-default"})
  for i in anlamlari:
    kelimeninAnlami = i.find_all("div", attrs = {"class": "article-content"})
  
  for anlam in anlamlari:
    kanlam = anlam.find_all("strong")
  
  anlamlariList = []
  for i in range(len(kanlam)):
    anlamlariList.append(kanlam[i].text)
  
  return anlamlariList

In [ ]:
kelimeninAnlamlariList = AranilanKelimeninAnlamlari(sozluk)
data = pd.Series(kelimeninAnlamlariList)

In [ ]:
data

0    Al, kızıl renk
1    Bu renkte olan
dtype: object

In [ ]:
sozluk2 = input("Lutfen kelimeyi giriniz: ")
sozluk2 = sozluk2.replace(" ", "-")
sozluk2 = sozluk2.lower()

Lutfen kelimeyi giriniz: al


In [ ]:
kelimeninAnlamlariList2 = AranilanKelimeninAnlamlari(sozluk2)
data2 = pd.Series(kelimeninAnlamlariList2)

In [ ]:
data2

0            Kanın rengi, kızıl, kırmızı
1                         Bu renkte olan
2    Dorunun açığı, kızıla çalan at donu
3                  Bu renkte olan "(at)"
4      Yüze sürülen pembe düzgün, allık.
5         Alüminyum elementinin simgesi.
6           Aldatma, düzen, tuzak, hile.
dtype: object

In [ ]:
sozluk2

'al'

# Kelime benzerliği tespiti

- Data Preprocessing

1. Cleaning
2.Tokenization
3. Stop words removal
4. Lemmatization
5. Stemming

In [ ]:
pip install zeyrek

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('turkish'))

# Lemmatization
import zeyrek
analyzer = zeyrek.MorphAnalyzer()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# NLP

## Cleaning
- Convert words to lowercase.
- remove numbers.
- remove not string(é!'^+%...).
- Stop to words

In [ ]:
wordsArr1 = []
wordsArrStoptoWord = []
wordsArr = []
wordl = []
wordsL = []
wordsLemma = []

for i in range(len(data)):
  # Data cleaning(re)
  words = data.iloc[i].lower()
  words = re.sub(r'[~^0-9]', '', words)
  words = re.sub(r'[^\w]', ' ', words)

  # Tokenization
  words = word_tokenize(words)
  wordsArr1.append(words)
  
  # Stop to word
  wordsFiltered = []
  for w in words:
    if w not in stopWords:
      wordsFiltered.append(w)
    wordsArrStoptoWord.append(wordsFiltered)

# Listedeki aynı kelimeleri kaldırdım
for i in wordsArrStoptoWord:
  if i not in wordsArr:
    wordsArr.append(i)

# Lemmatization
for l in wordsArr:
  for i in l:
    wordl.append(analyzer.lemmatize(i))
  wordsL.append(wordl)

for i in wordsL:
  if i not in wordsLemma:
    wordsLemma.append(i)

# Words Lemma
kelime = []
lemmaKelime = []

for i in range(len(wordsLemma[0])):
  k = wordsLemma[0][i - 1]
  k = str(k)
  k = re.sub(r'[^\w]', ' ', k)
  k = k.split()

  kelime.append(k[0])
  lemmaKelime.append(" , ".join(k[1:]))

KelimeDataString = sozluk + " Kelime"
kelimeData = pd.DataFrame(kelime, columns = [KelimeDataString])
LemmakelimeData = pd.DataFrame(lemmaKelime, columns = ["Lemma Kelime"])
data = pd.concat([kelimeData, LemmakelimeData], axis = 1)

In [ ]:
data

,kırmızı Kelime,Lemma Kelime
0,olan,"o , olmak"
1,al,"al , Al , almak"
2,kızıl,"kızıl , kızmak"
3,renk,renk
4,renkte,renk
